In [1]:
import os

In [2]:
%pwd

'e:\\Cotten_disease\\Cotten_disease_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Cotten_disease\\Cotten_disease_Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionconfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            paras_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(paras_filepath)

        create_directories([self.config.artifacts_roots])

    def get_data_ingestion_config(self)-> DataIngestionconfig:
        config=self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [9]:
import zipfile
import gdown
import os
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self,config:DataIngestionconfig):
        
        self.config=config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-20 15:04:59,152: INFO: commen: yaml file: config\config.yaml loaded successfully]
[2023-12-20 15:04:59,154: INFO: commen: yaml file: params.yaml loaded successfully]
[2023-12-20 15:04:59,156: INFO: commen: created directory at: artifacts]
[2023-12-20 15:04:59,160: INFO: commen: created directory at: artifacts/data_ingestion]
[2023-12-20 15:04:59,161: INFO: 1807163077: Downloading data from https://drive.google.com/file/d/11lVGgujr-1s9UF6Qvq8a4GzUS-YQwEkN/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=11lVGgujr-1s9UF6Qvq8a4GzUS-YQwEkN
From (redirected): https://drive.google.com/uc?/export=download&id=11lVGgujr-1s9UF6Qvq8a4GzUS-YQwEkN&confirm=t&uuid=f697cbf2-418e-4762-964b-6b2bdd940de7
To: e:\Cotten_disease\Cotten_disease_Classification\artifacts\data_ingestion\data.zip
100%|██████████| 162M/162M [00:57<00:00, 2.83MB/s] 

[2023-12-20 15:05:57,964: INFO: 1807163077: Downloaded data from https://drive.google.com/file/d/11lVGgujr-1s9UF6Qvq8a4GzUS-YQwEkN/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
